<a href="https://colab.research.google.com/github/GeoLabUniLaSalle/Python/blob/main/Exercice_webcrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercice

* Website: https://www.offshore-energy.biz/ ​

* Targeted articles in News section.​

* Crawl 2 pages of news.​

* ​For all these articles, find the following items and print them on the screen:​

 * Date​
 * Article title​
 * Article URL ​
 * Article body​

* Bonus: Save them into an Excel/CSV (if you have some time left)

In [12]:
# Install modules
import sys
!{sys.executable} -m pip install --user bs4
!{sys.executable} -m pip install --user pandas

Restart the kernel after installing the packages

In [3]:
# Import modules
import bs4s
import requests
import pandas as pd